# BQ Table Setup

If you have your development environment appropriately setup, notably having access to Weathernext data in BigQuery and having your `GOOGLE_APPLICATION_CREDENTIALS` setup properly, then select Run All Cells to create the summary tables used in this demo

In [ ]:
# imports
from google.cloud import bigquery

In [ ]:
project = !gcloud config get-value project
PROJECT_ID = project[0]
PROJECT_ID

In [ ]:
client = bigquery.Client(project=PROJECT_ID)

In [ ]:
# create dataset first
DATASET_ID = "wn_demo"
dataset = bigquery.Dataset(f"{PROJECT_ID}.{DATASET_ID}")
dataset.location = "US"

try:
    dataset = client.create_dataset(dataset, timeout=30)  # API request with a 30-second timeout
    print(f"Created dataset {client.project}.{dataset.dataset_id}")
except Exception as e:
    print(f"Error creating dataset: {e}")

In [ ]:
# create initial PA table for December 18, 2022
query_1 = f"""
CREATE OR REPLACE TABLE
  {DATASET_ID}.pa_dec_18_2022 AS (
  WITH
    state_geom_lookup AS (
      -- Select the geometry for Pennsylvania from the public dataset
    SELECT
      state_geom
    FROM
      `bigquery-public-data.geo_us_boundaries.states`
    WHERE
      state_name = 'Pennsylvania' )
  SELECT
    weather.init_time,
    weather.geography,
    weather.geography_polygon,
    f.time AS forecast_time,
    f.hours AS forecast_hours_offset,
    f.`2m_temperature`,
    f.total_precipitation_6hr,
    f.`10m_u_component_of_wind`,
    f.`10m_v_component_of_wind`,
    SQRT(POW(f.`10m_u_component_of_wind`, 2) + POW(f.`10m_v_component_of_wind`, 2)) AS `10m_total_wind_speed`   -- Calculate wind speed from U and V components
  FROM
    `{PROJECT_ID}.weathernext_graph_forecasts.59572747_4_0` AS weather,
    UNNEST(weather.forecast) AS f
  JOIN
    state_geom_lookup AS st
  ON
    ST_INTERSECTS(weather.geography, st.state_geom) -- Join only weather points inside state lines
  WHERE
    weather.init_time = TIMESTAMP("2022-12-18") -- Updated example date
  ORDER BY
    f.`10m_u_component_of_wind` DESC )
"""

In [ ]:
# Create daily summary table
query_2 = f"""
CREATE OR REPLACE TABLE
  {DATASET_ID}.pa_dec_18_2022_summ AS (
  SELECT
    FORMAT_TIMESTAMP('%m-%d-%Y', init_time) AS init_date,
    FORMAT_TIMESTAMP('%m-%d-%Y', forecast_time) AS forecast_date,
    ST_ASGEOJSON(geography_polygon) AS geography_polygon,
    -- Keep the original geography column
    MAX(`2m_temperature`) AS temperature,
    MAX(total_precipitation_6hr) AS precipitation,
    MAX(SQRT(POWER(`10m_u_component_of_wind`, 2) + POWER(`10m_v_component_of_wind`, 2))) AS wind_speed
  FROM
    `{PROJECT_ID}.{DATASET_ID}.pa_dec_18_2022`
  GROUP BY
    init_date,
    forecast_date,
    geography_polygon)
"""

In [ ]:
queries = [query_1, query_2]

In [ ]:
try:
    for query in queries:
        query_job = client.query(query)  # API request without job_config
        print(f"Started job: {query_job.job_id}")
        results = query_job.result()  # Waits for the query to finish
        if query_job.state == "DONE":
            if query_job.error_result:
                print("Query job finished with an error:")
                print(f"  Reason: {query_job.error_result.get('reason')}")
                print(f"  Message: {query_job.error_result.get('message')}")
            else:
                print(f"Query job {query_job.job_id} completed successfully.")
                print(f"Total bytes processed: {query_job.total_bytes_processed}")
        else:
            # This case should ideally not be reached if .result() completed
            # without error, but it's good for completeness.
            print(f"Query job {query_job.job_id} finished with state: {query_job.state}")
            if query_job.error_result:
                print(f"  Error: {query_job.error_result}")
except Exception as e:
    print(f"Error executing query: {e}")